In [34]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import BucketedRandomProjectionLSH

# Initialize Spark session for Arbitrage Opportunities Detection with necessary configurations
spark = SparkSession \
    .builder \
    .appName("ArbitrageOpportunitiesDetection") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-mllib_2.12:3.0.1") \
    .getOrCreate()


24/04/21 10:55:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [42]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# Initialize Spark session for Arbitrage Opportunities Detection with necessary configurations
spark = SparkSession \
    .builder \
    .appName("ArbitrageOpportunitiesDetection") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-mllib_2.12:3.0.1") \
    .getOrCreate()

# Define the schema based on the output from the sample file
schema = StructType([
    StructField("symbol_id", StringType(), True),
    StructField("time_exchange", LongType(), True),  # Epoch time in milliseconds
    StructField("price", DoubleType(), True),
    StructField("uuid", StringType(), True),
    StructField("taker_side", StringType(), True)
])


# Define the directory to monitor for new JSON files
input_directory = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/temp_folder/"

# Set up the DataFrame to read streaming data from the directory
streaming_df = spark \
    .readStream \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .json(input_directory)

# Convert time_exchange from epoch to timestamp if necessary
from pyspark.sql.functions import from_unixtime
streaming_df = streaming_df.withColumn("time_exchange", from_unixtime(col("time_exchange") / 1000))

# Display the processed stream
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


24/04/21 11:20:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/04/21 11:20:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-e6091573-4d93-4dd2-89bb-9d2bf38adc4d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/21 11:20:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 217
-------------------------------------------
+-------+-------------------+----------+-------------------+--------------------+
|  price|          symbol_id|taker_side|      time_exchange|                uuid|
+-------+-------------------+----------+-------------------+--------------------+
|64800.1|KRAKEN_SPOT_BTC_USD|       BUY|2024-04-21 04:20:05|9743430a5b87fd1f7...|
+-------+-------------------+----------+-------------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-------------------+-------+--------------------+----------+
|           symbol_id|      time_exchange|  price|                uuid|taker_side|
+--------------------+-------------------+-------+--------------------+----------+
|BITFINEX_SPOT_BTC...|2024-04-21 04:20:07|64749.0|8cae24eb23f8860df...|      SELL|
+--------------------+-------------------+-------+-----------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [40]:
# Path to the directory containing JSON files
json_files_directory = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/temp_folder/"

# Append the filename correctly with a slash
sample_file_path = json_files_directory + "batch_1_0.json"

# Read one file to infer schema
sample_df = spark.read.json(sample_file_path)

# Print the schema to use it in the streaming read
sample_df.printSchema()


root
 |-- price: double (nullable = true)
 |-- symbol_id: string (nullable = true)
 |-- taker_side: string (nullable = true)
 |-- time_exchange: long (nullable = true)
 |-- uuid: string (nullable = true)



In [8]:
spark.stop()

### Final code to run for applying K-Means, LSH and final arbitrage calculation logic(extract every five minute lowest price for buy and highest price for sell among all three exchanges and calculate price difference and add as anew column)

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, unix_timestamp, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, BucketedRandomProjectionLSH
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import lit

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("ArbitrageOpportunitiesDetection") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Define the schema for the incoming data
schema = StructType([
    StructField("symbol_id", StringType(), True),
    StructField("time_exchange", LongType(), True),
    StructField("price", DoubleType(), True),
    StructField("uuid", StringType(), True),
    StructField("taker_side", StringType(), True)
])

# Directory containing the JSON files
input_directory = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/temp_folder"

# Read streaming data
streaming_df = spark.readStream \
    .schema(schema) \
    .json(input_directory) \
    .withColumn("timestamp", from_unixtime(col("time_exchange") / 1000)) \
    .withColumn("time_minute", (unix_timestamp("timestamp") / 60).cast("integer"))

# Assemble features for K-Means and LSH
vecAssembler = VectorAssembler(inputCols=["price"], outputCol="price_features")
scaler = StandardScaler(inputCol="price_features", outputCol="scaled_price_features")
kmeans = KMeans(featuresCol="scaled_price_features", predictionCol="time_cluster", k=5, seed=42)
brp = BucketedRandomProjectionLSH(inputCol="scaled_price_features", outputCol="hashes", bucketLength=0.1, numHashTables=3)

# Combine all stages into a pipeline
pipeline = Pipeline(stages=[vecAssembler, scaler, kmeans, brp])

from pyspark.sql.functions import lit

def process_batch(df, epoch_id):
    # Fit the pipeline model and transform the DataFrame
    if df.count() > 0:
        model = pipeline.fit(df)
        transformed_df = model.transform(df)

        # Aggregate to find minimum buy price and maximum sell price within each cluster
        min_prices = transformed_df.filter(col("taker_side") == "BUY").groupBy("time_cluster").agg(spark_min("price").alias("min_buy_price"))
        max_prices = transformed_df.filter(col("taker_side") == "SELL").groupBy("time_cluster").agg(spark_max("price").alias("max_sell_price"))

        # Join min and max prices with original data
        result_df = transformed_df.join(min_prices, "time_cluster", "left_outer").join(max_prices, "time_cluster", "left_outer")

        # Calculate price difference
        result_df = result_df.withColumn("price_diff", col("max_sell_price") - col("min_buy_price"))

        # Fill in missing values with defaults
        default_values = {
            "min_buy_price": 0.0,  # Default value for min_buy_price if missing
            "max_sell_price": 0.0,  # Default value for max_sell_price if missing
            "price_diff": 0.0,  # Default value for price_diff if missing
            # Include defaults for any other fields as necessary
        }
        result_df = result_df.na.fill(default_values)

        # Select necessary columns and write to JSON
        result_df.select(
            "symbol_id", "time_exchange", "timestamp", "uuid", "taker_side", "price",
            "min_buy_price", "max_sell_price", "price_diff", "time_cluster", "hashes"
        ).write.mode('append').json("/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/last_folder/clusters.json")
    else:
        print(f"No data found in batch {epoch_id}. Will check again in 5 minutes.")


# Define the streaming query with a 5-minute processing time trigger
query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime="5 minutes") \
    .outputMode("update") \
    .start()

query.awaitTermination()


24/04/24 23:42:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/24 23:42:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-b3370f6d-fad4-425d-a85b-c2b2ae226e8f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/24 23:42:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/Users/priyasuresh/.pyenv/versions/3.10.4/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentra

Py4JError: An error occurred while calling o15751.awaitTermination

In [14]:
spark.stop()

24/04/25 00:10:00 ERROR MicroBatchExecution: Query [id = 0207d0fe-ac79-42cd-a0ae-3eef4f566758, runId = 02138f72-3e66-490b-9163-67226e4216dd] terminated with error
java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:8

### Final Code--logic changed

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, unix_timestamp, min as spark_min, max as spark_max
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, BucketedRandomProjectionLSH
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import lit

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("ArbitrageOpportunitiesDetection") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Define the schema for the incoming data
schema = StructType([
    StructField("symbol_id", StringType(), True),
    StructField("time_exchange", LongType(), True),
    StructField("price", DoubleType(), True),
    StructField("uuid", StringType(), True),
    StructField("taker_side", StringType(), True)
])

# Directory containing the JSON files
input_directory = "/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/temp_folder"

# Read streaming data
streaming_df = spark.readStream \
    .schema(schema) \
    .json(input_directory) \
    .withColumn("timestamp", from_unixtime(col("time_exchange") / 1000)) \
    .withColumn("time_minute", (unix_timestamp("timestamp") / 60).cast("integer"))

# Assemble features for K-Means and LSH
vecAssembler = VectorAssembler(inputCols=["price"], outputCol="price_features")
scaler = StandardScaler(inputCol="price_features", outputCol="scaled_price_features")
kmeans = KMeans(featuresCol="scaled_price_features", predictionCol="time_cluster", k=5, seed=42)
brp = BucketedRandomProjectionLSH(inputCol="scaled_price_features", outputCol="hashes", bucketLength=0.1, numHashTables=3)

# Combine all stages into a pipeline
pipeline = Pipeline(stages=[vecAssembler, scaler, kmeans, brp])

from pyspark.sql.functions import lit

def process_batch(df, epoch_id):
    # Fit the pipeline model and transform the DataFrame
    if df.count() > 0:
        model = pipeline.fit(df)
        transformed_df = model.transform(df)

        # Aggregate to find minimum buy price and maximum sell price within each cluster for each exchange
        min_prices = transformed_df.filter(col("taker_side") == "BUY").groupBy("symbol_id", "time_cluster").agg(spark_min("price").alias("min_buy_price"))
        max_prices = transformed_df.filter(col("taker_side") == "SELL").groupBy("symbol_id", "time_cluster").agg(spark_max("price").alias("max_sell_price"))

        # Join min and max prices with original data
        result_df = transformed_df.join(min_prices, ["symbol_id", "time_cluster"], "left_outer").join(max_prices, ["symbol_id", "time_cluster"], "left_outer")

        # Calculate price difference
        result_df = result_df.withColumn("price_diff", col("max_sell_price") - col("min_buy_price"))

        # Fill in missing values with defaults
        default_values = {
            "min_buy_price": 0.0,  # Default value for min_buy_price if missing
            "max_sell_price": 0.0,  # Default value for max_sell_price if missing
            "price_diff": 0.0,  # Default value for price_diff if missing
        }
        result_df = result_df.na.fill(default_values)

        # Select necessary columns and write to JSON
        result_df.select(
            "symbol_id", "time_exchange", "timestamp", "uuid", "taker_side", "price",
            "min_buy_price", "max_sell_price", "price_diff", "time_cluster", "hashes"
        ).write.mode('append').json("/Users/priyasuresh/Documents/Documents_Priya’s MacBook Pro/SPRING2024/Data 228-Big Data/project/last_folder/clusters.json")
        
    else:
        print(f"No data found in batch {epoch_id}. Will check again in 5 minutes.")


# Define the streaming query with a 5-minute processing time trigger
query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime="5 minutes") \
    .outputMode("update") \
    .start()

query.awaitTermination()


24/04/27 12:13:58 WARN Utils: Your hostname, Priyas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.79 instead (on interface en0)
24/04/27 12:13:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/27 12:13:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/27 12:13:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/27 12:14:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/9j/0m5g2l6d03z4pd5y6pt7xh1h0000gn/T/temporary-520d9a0d-b1a3-441a-910f-9526561945c7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important

Py4JError: An error occurred while calling o58.awaitTermination

In [2]:
spark.stop()